##### Each panel should have a date and hour. Some records will have Null for population: impute those records

In [1]:
import pandas as pd
import numpy  as np
from config import getClient
from datetime import datetime, timedelta
import time
from itertools import groupby
import math
from os import listdir
from os.path import isfile, join

In [2]:
dataloc = getClient("OutFront")

In [3]:
panels = [f for f in listdir(dataloc+"panels/") if isfile(join(dataloc+"panels/", f))]

print("{:,.0f} panels to process".format(len(panels)))

699 panels to process


In [4]:
notFound = open(dataloc+"notFound.csv", "w")

In [5]:
# Create list of panel/date with missing hours
def getMissing(grp):
    missingList = []
    for idx, val in grp:
        missing = val.loc[val["population"].isnull()]["hour"]
        if not missing.empty:
            missingList.append(idx)
    return missingList

In [6]:
def writeNF(panel, dt):
    rec = panel+","+str(dt)+"\n"
    notFound.write(rec)

In [7]:
def copyWeek(df, panel, date, hour, numNaN):
    try:
        dt = date - timedelta(days=7)
        start = df.index.get_loc((panel, dt, hour))
        found = True
    except:
        try:
            dt    = date + timedelta(days=7)
            start = df.index.get_loc((panel, dt, hour))
            found = True
        except:
            writeNF(panel, dt)
            found = False
    if found:
        end   = start + numNaN
        pops  = df.iloc[start:end]["population"]
    
        for hr in range(hour, hour+numNaN):
            df.loc[panel,date,hr] = pops.loc[panel, dt, hr]
    return df

In [8]:
def sameDay(df, panel, date, hour, numNaN):
    tmp = df.loc[panel, date]
    pops = tmp["population"]
    if hour == 0:
        dt = date - timedelta(days=1)
        try:
            tmp = df.loc[panel, dt]
            found = True
            avg = (tmp.iloc[23] + pops[hour+numNaN])/2
        except:
            writeNF(panel, dt)
            found = False
        if found:
            if avg[0] > 1:
                for hr in range(hour, hour+numNaN):
                    df.loc[panel, date, hr] = avg
    elif (hour+numNaN) > 23:
        dt = date + timedelta(days=1)
        try:
            tmp = df.loc[panel, dt]
            found = True
        except:
            writeNF(panel, dt)
            found = False
        if found:
            nextDayPop = np.nan
            n = 0
            while math.isnan(nextDayPop):
                nextDayPop = tmp.iloc[n][0]
                n += 1
                if n > 23:
                    break
            if nextDayPop > 1:
                avg =  (nextDayPop + pops[hour-1]) / 2
                for hr in range(hour, 24):
                    df.loc[panel, date, hr] = avg
            else:
                writeNF(panel, dt)
    else:
        avg = (pops[hour-1] + pops[hour+numNaN]) / 2
        if avg > 1:
            for hr in range(hour, hour+numNaN):
                df.loc[panel, date, hr] = avg
    return df

In [9]:
# "clusters" is sometimes zero-indexed and sometimes 1, for no known
# reason. It screws up getting the hour, so this will compensate
def processClusters(df, panel, date, clusters):
    offset = clusters.index[0]   # value will be either 0 or 1

    count = 0
    for numNaN in clusters:
        if numNaN in [1,2,3]:
            hour = count + offset
            df = sameDay(df, panel, date, hour, numNaN)
        elif numNaN > 3:
            hour = count + offset
            df = copyWeek(df, panel, date, hour, numNaN)
        offset += numNaN            
        count += 1
    return df

In [10]:
# input is an entire panel (all dates)
def processPanel(df):
    df  = df.set_index(["panel", "date"])
    grp = df.groupby(level=[0,1])
    df  = df.reset_index()
    df  = df.set_index(["panel", "date", "hour"])

    # Get a list of missing values for each panel/date
    missingList = getMissing(grp)

    # For each panel/date combination, process the NaN values
    for x in missingList:
        panel = x[0]
        date  = x[1]
        tmp = df.loc[panel, date]["population"]
        clusters = tmp.isnull().astype(int).groupby(tmp.notnull().astype(int).cumsum()).sum()
        df = processClusters(df, panel, date, clusters)
    return df

In [ ]:
count = 0
start = time.time()

for panel in panels:
    df = pd.read_csv(dataloc+"panels/"+panel,\
                     dtype={"hour":'int8',\
                            "population":'int32'})
    df = processPanel(df)
    df.to_csv(dataloc+"panels/"+panel)
    
notFound.close()
end = time.time()
print("Completed after {:.0f} minutes".format((end-start)/60))